In [1]:
from pathlib import Path
import pandas as pd


In [2]:
# read CSV and create Pandas dataframe
df = pd.read_csv("trip_data.csv")

# preview df
df

,request_datetime,hvfhs_license_num,dispatching_base_num,originating_base_num,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,2023-01-01 00:40:07,HV0003,B03404,B03404,2023-01-01 00:58:14,2023-01-01 00:58:36,2023-01-01 01:27:56,3,116,9.500,...,3.48,0.00,0.0,0.00,34.47,N,N,,N,N
1,2023-01-01 02:11:51,HV0003,B03404,B03404,2023-01-01 02:22:37,2023-01-01 02:24:37,2023-01-01 02:35:54,244,244,0.740,...,1.89,0.00,0.0,0.00,14.87,N,N,,N,Y
2,2023-01-01 06:42:32,HV0003,B03404,B03404,2023-01-01 06:47:01,2023-01-01 06:47:34,2023-01-01 07:20:43,165,230,13.730,...,4.43,2.75,0.0,0.00,34.12,N,N,,N,N
3,2023-01-01 14:00:21,HV0003,B03404,B03404,2023-01-01 14:06:37,2023-01-01 14:06:51,2023-01-01 14:23:17,72,61,2.650,...,1.50,0.00,0.0,0.00,15.69,N,N,,N,N
4,2023-01-01 14:04:45,HV0003,B03404,B03404,2023-01-01 14:06:18,2023-01-01 14:06:45,2023-01-01 14:22:28,230,161,0.890,...,2.65,2.75,0.0,0.00,23.83,N,N,,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,2023-12-29 21:23:12,HV0003,B03404,B03404,2023-12-29 21:23:38,2023-12-29 21:24:50,2023-12-29 21:37:12,69,74,3.510,...,1.43,0.00,0.0,0.00,11.97,N,N,N,N,N
1196,2023-12-30 06:17:30,HV0005,B03406,NaN,NaN,2023-12-30 06:22:47,2023-12-30 06:33:39,25,181,1.906,...,1.08,0.00,0.0,0.00,8.71,N,N,N,N,N
1197,2023-12-31 08:41:34,HV0005,B03406,NaN,NaN,2023-12-31 08:43:54,2023-12-31 08:49:55,108,21,1.450,...,0.00,0.00,0.0,0.00,7.97,N,N,N,N,N
1198,2023-12-31 20:48:47,HV0005,B03406,NaN,NaN,2023-12-31 20:53:59,2023-12-31 21:05:30,260,56,2.398,...,1.41,0.00,0.0,3.54,12.73,N,N,N,N,N


In [3]:
# df info
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   request_datetime      1200 non-null   object 
 1   hvfhs_license_num     1200 non-null   object 
 2   dispatching_base_num  1200 non-null   object 
 3   originating_base_num  871 non-null    object 
 4   on_scene_datetime     871 non-null    object 
 5   pickup_datetime       1200 non-null   object 
 6   dropoff_datetime      1200 non-null   object 
 7   PULocationID          1200 non-null   int64  
 8   DOLocationID          1200 non-null   int64  
 9   trip_miles            1200 non-null   float64
 10  trip_time             1200 non-null   int64  
 11  base_passenger_fare   1200 non-null   float64
 12  tolls                 1200 non-null   float64
 13  bcf                   1200 non-null   float64
 14  sales_tax             1200 non-null   float64
 15  congestion_surcharge 